# Case 2. Тарифы телекоммуникационного провайдера

Абонент хочет оптимизировать свои затраты на связь, сменив текущий тарифный план. Три оператора сотовой связи прислали свои предложения: описание всех тарифных планов агрегированы в единый документ. Имеется также выгрузка исходящих вызовов абонента за последние 6 месяцев.

Необходимо исследовать характер исходящих звонков абонента и предложить оптимальный тариф, обосновав свои рекомендации аналитическими расчетами.

Примечание: начать исследование рекомендуется с описания начального набора данных.


In [1]:
import pandas as pd

In [5]:
d = pd.read_excel("C:\\Users\\HOME\\PycharmProjects\\case_studies_da\\2. Telecom\\telecom_data.xlsx", engine='openpyxl')
d.head(10)

,Дата звонка,Длительность разговора,Оператор связи вызываемого абонента
0,15.12.2010 10:50:54,00:01:49,билайн
1,15.12.2010 11:50:42,00:00:57,билайн
2,30.11.2010 16:39:34,00:00:55,билайн
3,30.11.2010 16:39:34,00:00:55,билайн
4,01.12.2010 09:49:09,00:06:47,билайн
5,01.12.2010 14:01:02,00:01:14,билайн
6,02.12.2010 11:28:55,00:00:38,билайн
7,02.12.2010 11:40:47,00:00:31,билайн
8,02.12.2010 11:51:17,00:00:11,билайн
9,02.12.2010 12:00:16,00:00:45,билайн
